In [1]:
# !pip3 install openai
!pip3 install os
!pip3 install pandas
!pip3 install sqlalchemy
!pip install openai==0.28

import openai
import os
import pandas as pd
from sqlalchemy import create_engine, text

os.environ["API_KEY"] = "<OPENAI_API_KEY>"
openai.api_key = os.getenv("API_KEY")

# print(openai.api_key)

ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [2]:
csv_file_path = '/Users/anshul/_learning/ai/coursera-aiSQLQueries/customer_orders.csv'
data_frame = pd.read_csv(csv_file_path)
# print(data_frame)
# print(data_frame.head())
# print(data_frame.dtypes)
print(data_frame.columns)

Index(['ORDERNUMBER', 'QUANTITYORDERED', 'PRICEEACH', 'ORDERLINENUMBER',
       'SALES', 'ORDERDATE', 'STATUS', 'QTR_ID', 'MONTH_ID', 'YEAR_ID',
       'PRODUCTLINE', 'MSRP', 'PRODUCTCODE', 'CUSTOMERNAME', 'PHONE',
       'ADDRESSLINE1', 'ADDRESSLINE2', 'CITY', 'STATE', 'POSTALCODE',
       'COUNTRY', 'TERRITORY', 'CONTACTLASTNAME', 'CONTACTFIRSTNAME',
       'DEALSIZE'],
      dtype='object')


In [3]:
engine = create_engine('sqlite:///:memory:', echo=True)
data_frame.to_sql("SalesTable", engine)

2024-09-24 11:44:36,883 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-24 11:44:36,886 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SalesTable")
2024-09-24 11:44:36,886 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-24 11:44:36,887 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("SalesTable")
2024-09-24 11:44:36,887 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-09-24 11:44:36,888 INFO sqlalchemy.engine.Engine 
CREATE TABLE "SalesTable" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"TERRITORY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT, 
	"DEALSI

2823

In [4]:
def execute_sql_query(connection, query):
    result_obj = connection.execute(text(query))
    return result_obj.fetchall()

In [5]:

with engine.connect() as conn:
    sales_sum_result = execute_sql_query(conn, "Select sum(sales) from SalesTable")
    print(f"Sum of sales: {sales_sum_result}")

2024-09-24 11:44:36,927 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-24 11:44:36,927 INFO sqlalchemy.engine.Engine Select sum(sales) from SalesTable
2024-09-24 11:44:36,928 INFO sqlalchemy.engine.Engine [generated in 0.00072s] ()
Sum of sales: [(10032628.85,)]
2024-09-24 11:44:36,928 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
def table_schema_prompt(df):
    columns = ", ".join(df.columns)
    return f"### SQLite Table Schema: \n# SalesTable({columns}) \n"

def get_user_input():
    return input("Enter your query: ")

def full_prompt(df, user_prompt):
    schema_prompt = table_schema_prompt(df)
    return f"{schema_prompt}### Query to answer: {user_prompt}\n SELECT"

user_prompt = get_user_input()
complete_prompt = full_prompt(data_frame, user_prompt)

print(complete_prompt)



Enter your query:  get sum of all sales


### SQLite Table Schema: 
# SalesTable(ORDERNUMBER, QUANTITYORDERED, PRICEEACH, ORDERLINENUMBER, SALES, ORDERDATE, STATUS, QTR_ID, MONTH_ID, YEAR_ID, PRODUCTLINE, MSRP, PRODUCTCODE, CUSTOMERNAME, PHONE, ADDRESSLINE1, ADDRESSLINE2, CITY, STATE, POSTALCODE, COUNTRY, TERRITORY, CONTACTLASTNAME, CONTACTFIRSTNAME, DEALSIZE) 
### Query to answer: get sum of all sales
 SELECT


In [12]:
response = openai.Completion.create(
    model="gpt-3.5-turbo-instruct",
    prompt=complete_prompt,
    temperature=0,
    max_tokens=150
)

def process_repsonse(res):
    raw_query = res['choices'][0]['text'].strip()
    if not raw_query.upper().startswith("SELECT"):
        raw_query = f"SELECT {raw_query}"
    return raw_query
    

In [19]:
final_query = process_repsonse(response)

In [20]:
with engine.connect() as conn:
    final_results = execute_sql_query(conn, final_query)
print(f"Final results: {final_results}")

2024-09-24 11:53:15,653 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-09-24 11:53:15,654 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM SalesTable;
2024-09-24 11:53:15,654 INFO sqlalchemy.engine.Engine [generated in 0.00187s] ()
2024-09-24 11:53:15,661 INFO sqlalchemy.engine.Engine ROLLBACK
Final results: [(10032628.85,)]
